Collaborators: 
    Zach Chommala
    Bao Nguyen
    Alex Ho

# CSCI 3155: Assignment 3 

Topics covered: ASTs, pattern matching, higher order functions.

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

__Jonathan Phouminh__

## Problem 1 (30 points)

### 1A (10 points) 

This question asks you to implement the following grammar for a very simple programming language in scala.  The programming language allows you to declare variables and assign them values.

Example Program:

~~~
x1 := 10 # create a variable x1 and assign 10 to the variable x1.
x2 := 20 + x1 * x1 # create a variable x2 and assign 20 + x1 * x1
x3 := x1 + x2 # create a variable x3 and assign x1 + x2
x4 := x3 # create a variable x4 and assign x3 to it

(x1 + x2 + x3) # Final Return Expression that is the value of the whole program.
~~~

Please use constructors with the same name as the nonterminals in the grammar.



$$\newcommand\nt[1]{\textbf{#1}}$$
$$ \begin{array}{rcl}
\nt{Expr} & \Rightarrow & Plus(\nt{Expr}, \nt{Expr})\\
&|&  Minus(\nt{Expr}, \nt{Expr})\\
&|& Star(\nt{Expr}, \nt{Expr})\\
&|& Var(\nt{String}) \\
&|& Const(\nt{Double}) \\[10pt]
\nt{Statement} & \Rightarrow & Assignment(\nt{String}, \nt{Expr})\\[5pt]
\nt{Program} & \Rightarrow &  CalcProgram( \nt{Statement}* ,\  \nt{Expr})\\[5pt]
\end{array}$$


Complete the scala definition for the grammar above. Please use scala datatypes for Double and String.
Also, please use List(..) whenever Kleene star is used.


In [2]:
sealed trait Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Star(e1: Expr, e2: Expr) extends Expr
case class Var(s: String) extends Expr
case class Const(d: Double) extends Expr

sealed trait Statement
case class Assignment(s: String, e: Expr) extends Statement

sealed trait Program 
case class CalcProgram(statments: List[Statement], e: Expr) extends Program 

defined trait Expr
defined class Plus
defined class Minus
defined class Star
defined class Var
defined class Const
defined trait Statement
defined class Assignment
defined trait Program
defined class CalcProgram

In [3]:
//BEGIN TEST
val v1 = Assignment("x", Const(2.0))
val v2 = Assignment("y", Plus(Var("x"), Const(4.0)))
val v3 = Assignment("z", Var("y"))
passed(5)
//END TEST


*** Tests Passed (5 points) ***


v1: Assignment = Assignment("x", Const(2.0))
v2: Assignment = Assignment("y", Plus(Var("x"), Const(4.0)))
v3: Assignment = Assignment("z", Var("y"))

In [4]:
//BEGIN TEST
val v1 = Assignment("x", Const(2.0))
val v2 = Assignment("y", Plus(Var("x"), Const(4.0)))
val v3 = Assignment("z", Var("y"))
val p1 = CalcProgram(List(v1, v2, v3), Var("z"))
val p2 = CalcProgram(List(), Var("y"))

passed(5)
//END TEST


*** Tests Passed (5 points) ***


v1: Assignment = Assignment("x", Const(2.0))
v2: Assignment = Assignment("y", Plus(Var("x"), Const(4.0)))
v3: Assignment = Assignment("z", Var("y"))
p1: CalcProgram = CalcProgram(
  List(
    Assignment("x", Const(2.0)),
    Assignment("y", Plus(Var("x"), Const(4.0))),
    Assignment("z", Var("y"))
  ),
  Var("z")
)
p2: CalcProgram = CalcProgram(List(), Var("y"))

### Part 1B (10 points)

In class we saw how to evaluate an expression given a map from identifiers to their values. 

~~~
def evalExpr(e: Expr, env: Map[String, Double]): Double = ...
~~~

In this assignment,  you are asked to handle assignment statement. The idea here is that the
assignment statement takes an environment and modifies it.

~~~
def evalStatement(s: Statement, env: Map[String, Double]): Map[String, Double] = ...
~~~

As an example consider the map  env

~~~
Map("x" -> 20, "y" -> 35)
~~~

The assignment statement z := x + y

~~~
Assignment("z", Plus(Var("x"), Var("y")))
~~~

Should return a map 

~~~
Map("x"-> 20, "y" -> 35, "z" -> 55)
~~~

The strategy should be as follows: (a) evaluate the expression in the RHS of the assignment statement, 
(b) add the entry that maps the declared variable to the new value and (c) return this map.

https://docs.scala-lang.org/overviews/collections-2.13/maps.html



In [5]:
def evalExpr(e: Expr, env: Map[String, Double]): Double = {
    def binFun(e1: Expr, e2: Expr, op: (Double, Double) => Double): Double = {
        val v1 = evalExpr(e1, env)
        val v2 = evalExpr(e2, env)
        op(v1, v2)
    }
    
    e match {
        case Plus(e1, e2) => binFun(e1, e2, _ + _ )
        case Minus(e1, e2) => binFun(e1, e2, _ - _ )
        case Star(e1, e2) => binFun(e1, e2, _ * _ )
        case Var(v) => {
            if (env.contains(v)){
                env(v)
            } else {
                throw new IllegalArgumentException(s"$v undefined variable.")
            }
        }
        case Const(f) => f
    }
}
/*
    You are given an assignment statement
        varName , this is the variable that you need to set a value for in the map 
        to get the value to map this you must evaluate rhs 
        then just add it to the map 
*/

def evalStatement(s: Statement, env: Map[String, Double]): Map[String, Double] = s match {
    case Assignment(varName, rhsExpr) => {
        val RHS = evalExpr(rhsExpr, env)
        env + (varName -> RHS)
        
    }
}


defined function evalExpr
defined function evalStatement

In [6]:
//BEGIN TEST
def testMap(m1: Map[String, Double], varname: String, testval: Double) = {
    m1.contains(varname) && m1(varname) == testval
}

val m1 = Map[String, Double]()
val v1 = Assignment("x", Const(2.0))
val m2 = evalStatement(v1, m1)
assert(testMap(m2, "x", 2.0), "TEST 1 PASSED!")

val v2 = Assignment("y", Plus(Var("x"), Const(4.0)))
val m3 = evalStatement(v2, m2)
assert(testMap(m3, "x", 2.0), "TEST 2.1 PASSED!")
assert(testMap(m3, "y", 6.0), "TEST 2.2 PASSED!")


val v3 = Assignment("z", Star(Var("y"), Var("x")))
val m4 = evalStatement(v3, m3)
assert(testMap(m4, "x", 2.0), "TEST 3.1 PASSED!")
assert(testMap(m4, "y", 6.0), "TEST 3.2 PASSED!")
assert(testMap(m4, "z", 12.0), "TEST 3.3 PASSED!")

passed(10)

//END TEST


*** Tests Passed (10 points) ***


defined function testMap
m1: Map[String, Double] = Map()
v1: Assignment = Assignment("x", Const(2.0))
m2: Map[String, Double] = Map("x" -> 2.0)
v2: Assignment = Assignment("y", Plus(Var("x"), Const(4.0)))
m3: Map[String, Double] = Map("x" -> 2.0, "y" -> 6.0)
v3: Assignment = Assignment("z", Star(Var("y"), Var("x")))
m4: Map[String, Double] = Map("x" -> 2.0, "y" -> 6.0, "z" -> 12.0)

### Part 1C (10 points)

Write an evaluator for a program (list of statements + return expression) that evaluates the program and returns the final value of the return expression after the statements in the program are run one by one. We explain how to evaluate a program step by step on an example program

~~~
x := 15 //Stmt1
y := x - 10 // Stmt 2
z := y + 5 // Stmt 3
x+ y+ z // Return expression
~~~

This program should return the value 30. The key idea is this:

a) Start with an empty environment map to begin with.
b) Execute each statement in the list of statements from beginning to end.
   When you execute the statement, do so with the current map and the return value is the next map.
c) Evaluate the return expression on the final map after you have executed all the statements.

Complete the code below. You can use loops and var for now. We will show you how to avoid it later.

Here is how you iterate through each element of a list in scala: 

~~~
for (item <- lst){
    // Do stuff with item
}
~~~

In [7]:
/*
PARAMETERS
    list of statements, then the return expression. 
    evaluate the function then return the final value of the return expression after the statements in the program 
    are done one by one. 
    
    1. Start with empty environemnt map 
    2. execute each statement in the list of statements 
    3. after each iteration, execute it with the current map, replace the entire map after execution
    4. evaulate the return expression on the final map after you have executed all the statements 
use these 
    sealed trait Expr
    case class Plus(e1: Expr, e2: Expr) extends Expr
    case class Minus(e1: Expr, e2: Expr) extends Expr
    case class Star(e1: Expr, e2: Expr) extends Expr
    case class Var(s: String) extends Expr
    case class Const(d: Double) extends Expr
    sealed trait Statement
    case class Assignment(s: String, e: Expr) extends Statement
*/
def evalExprHelper(e: Expr, env: Map[String, Double]): Double = {
    def binFun(e1: Expr, e2: Expr, op: (Double, Double) => Double): Double = {
        val v1 = evalExpr(e1, env)
        val v2 = evalExpr(e2, env)
        op(v1, v2)
    }
    
    e match {
        case Plus(e1, e2) => binFun(e1, e2, _ + _ )
        case Minus(e1, e2) => binFun(e1, e2, _ - _ )
        case Star(e1, e2) => binFun(e1, e2, _ * _ )
        case Var(v) => {
            if (env.contains(v)){
                env(v)
            } else {
                throw new IllegalArgumentException(s"$v undefined variable.")
            }
        }
        case Const(f) => f
    }
}

def evalStatementHelper(s: Statement, env: Map[String, Double]): Map[String, Double] = s match {
    case Assignment(varName, rhsExpr) => {
        val RHS = evalExpr(rhsExpr, env)
        env + (varName -> RHS)  // At this point the map should now be updated 
    }
}
def evalProgram(p: Program): Double = p match {
    case CalcProgram(lstOfStatement, returnExpr) => {
        var env: Map[String, Double] = Map()
        for(item <- lstOfStatement){
            env = evalStatementHelper(item,env)
        }
        evalExprHelper(returnExpr, env)
    }
}

defined function evalExprHelper
defined function evalStatementHelper
defined function evalProgram

In [8]:
//BEGIN TEST
val v1 = Assignment("x", Const(2.0))
val v2 = Assignment("y", Plus(Var("x"), Const(4.0)))
val v3 = Assignment("z", Var("y"))
val p1 = CalcProgram(List(v1, v2, v3), Var("z"))
print(evalProgram(p1))
assert(evalProgram(p1) == 6)
passed(5)
//END TEST

6.0
*** Tests Passed (5 points) ***


v1: Assignment = Assignment("x", Const(2.0))
v2: Assignment = Assignment("y", Plus(Var("x"), Const(4.0)))
v3: Assignment = Assignment("z", Var("y"))
p1: CalcProgram = CalcProgram(
  List(
    Assignment("x", Const(2.0)),
    Assignment("y", Plus(Var("x"), Const(4.0))),
    Assignment("z", Var("y"))
  ),
  Var("z")
)

In [9]:
//BEGIN TEST
val vv1 = Assignment("x", Const(2.0))
val vv2 = Assignment("y", Star(Var("x"), Var("x")))
val vv3 = Assignment("z", Star(Var("y"), Var("y")))
val vv4 = Assignment("w", Star(Var("z"), Var("z")))
val p2 = CalcProgram(List(vv1, vv2, vv3, vv4), Var("w"))
print(evalProgram(p2))
assert(evalProgram(p2) == 256)
passed(5)
//END TEST

256.0
*** Tests Passed (5 points) ***


vv1: Assignment = Assignment("x", Const(2.0))
vv2: Assignment = Assignment("y", Star(Var("x"), Var("x")))
vv3: Assignment = Assignment("z", Star(Var("y"), Var("y")))
vv4: Assignment = Assignment("w", Star(Var("z"), Var("z")))
p2: CalcProgram = CalcProgram(
  List(
    Assignment("x", Const(2.0)),
    Assignment("y", Star(Var("x"), Var("x"))),
    Assignment("z", Star(Var("y"), Var("y"))),
    Assignment("w", Star(Var("z"), Var("z")))
  ),
  Var("w")
)

## Problem 2 (20 points)

### A (5 points)
We defined lists in the class. Write a recursive procedure to get the nth element of the list or throw an `IllegalArgumentException` if $n < 0$ or $n >= \text{length of list} $. Assume $n=0$ obtains the very first element
and $n = \text{length of list} -1$ yields very last element. 

In [10]:
sealed trait NumList
case object Nil extends NumList
case class Cons(n: Int, l: NumList) extends NumList 

def countHelper(lst: NumList, n: Int, acc: Int = 0): Int = lst match{
    // this is just helping get the length of the given list
    case Nil => { 
        throw new IllegalArgumentException("exceeded size.")
    }
    // if we hit this case that means we have found the value and we are returning it 
    case Cons(k,_) if acc == n => {
        k
    }
    case Cons(j,innerList) => {
        countHelper(innerList,n, acc+1)
    }
}

// this is the 'main' function essentially
def getNthElement(lst: NumList, n: Int): Int = {
    if (n < 0 || lst == Nil){
        throw new IllegalArgumentException("how dare you")
    }
    println()
    val nth_element = countHelper(lst,n)
    return nth_element
}

defined trait NumList
defined object Nil
defined class Cons
defined function countHelper
defined function getNthElement

In [11]:
val l1 = Nil
val l2 = Cons(1, Cons(-1, Nil))
val l3 = Cons(1, Cons(2, l2))
val l4 = Cons(0, Cons(4, Cons(8, l3)))

val test1 = try {
    getNthElement(Nil, 3);
    assert(false, "Test 1 : getNthElement(Nil, 3) should raise an IllegalArgumentException. Your code did not.")
} catch {
    case e: IllegalArgumentException => "OK"
} 

assert(getNthElement(l2, 0) == 1, "Test2: getNthElement(l2, 0)  failed (expected answer = 1)")
assert(getNthElement(l3, 3) == -1, "Test3: getNthElement(l3, 3)  failed (expected answer = -1)")
assert(getNthElement(l4, 2) == 8, "Test4: getNthElement(l4, 2)  failed (expected answer = 8)")

val test2 = try {
    getNthElement(l4, 8);
    assert(false, "Test 5 : getNthElement(l4, 8) should raise an IllegalArgumentException. Your code did not.")
} catch {
    case e: IllegalArgumentException => "OK"
}

passed(5)






*** Tests Passed (5 points) ***


l1: Nil = Nil
l2: Cons = Cons(1, Cons(-1, Nil))
l3: Cons = Cons(1, Cons(2, Cons(1, Cons(-1, Nil))))
l4: Cons = Cons(0, Cons(4, Cons(8, Cons(1, Cons(2, Cons(1, Cons(-1, Nil)))))))
test1: Any = "OK"
test2: Any = "OK"

### B (7 points)
Write a recursive procedure that returns true if the list has the Fibonacci property. I.e, every element at position $i \geq 2$ is the sum of the two preceding elements. Note that the property is trivially true for lists of sizes $0$ and $1$.

In [12]:
// helper function that will return next element of a given list 
def getNextElement(lst: NumList): Int = lst match{
    case Nil => {-9999}
    case Cons(k,Nil) => {k}
    case Cons(k,_) => {k}
}

// function takes in the inner list of the first argument then the first element before that inner list 
def fib_helper(prev: Int, lst: NumList): Boolean = lst match{
    case Nil => {true}
    case Cons(k,Nil)=> {true}
    case Cons(k,lst) => {
        val sum = prev + k
        val nextVal = getNextElement(lst)
        if (sum != nextVal){
            return false
        }else{
            fib_helper(k,lst)
        }
    }
}
def isFibonacciList(lst: NumList): Boolean = lst match{
    case Nil => {true}      // handling the base cases 
    case Cons(_,Nil) => {true} 
    // giving it the first value then the rest of the list
    case Cons(k,lst) => {fib_helper(k,lst)}
}

defined function getNextElement
defined function fib_helper
defined function isFibonacciList

In [13]:
val l1 = Cons(12, Cons(25, Cons(37, Nil)))
assert(isFibonacciList(l1), 
       "Test case 1 :  isFibonacciList(l1) -- should return true")

val l2 = Cons(14, Cons(-1, Cons(13, l1 )))
assert(isFibonacciList(l2), 
       "Test case 2 :  isFibonacciList(l2) -- should return true")

val l3 = Cons(7, Cons(7, l2))

assert(!isFibonacciList(l3), 
       "Test case 3 :  isFibonacciList(l3) -- should return false")

val l4 = Cons(0, Cons(0, Cons(0, Cons(0, Cons(0, Cons(0, Nil))))))
assert(isFibonacciList(l2), 
       "Test case 4:  isFibonacciList(l4) -- should return true")

passed(7)


*** Tests Passed (7 points) ***


l1: Cons = Cons(12, Cons(25, Cons(37, Nil)))
l2: Cons = Cons(14, Cons(-1, Cons(13, Cons(12, Cons(25, Cons(37, Nil))))))
l3: Cons = Cons(
  7,
  Cons(7, Cons(14, Cons(-1, Cons(13, Cons(12, Cons(25, Cons(37, Nil)))))))
)
l4: Cons = Cons(0, Cons(0, Cons(0, Cons(0, Cons(0, Cons(0, Nil))))))

### C (8 points)
Write a recursive function `filterNumList(l: NumList, f: Int => Boolean): NumList` that takes in a `NumList` and a function `f: Int => Boolean`. 

1. It should return a new list that consist of all elements of the list `l` that return `true` when the function `f` is called on them.
2. The returned list elements must preserve the same order as in the original list.

In [14]:
sealed trait NumList
case object Nil extends NumList
case class Cons(n: Int, l: NumList) extends NumList 

def filterNumList(l: NumList, f: Int => Boolean): NumList = l match{
    case Nil => {
        Nil
    }
    case Cons(k,Nil) => {
        if (f(k) == true){
            Cons(k,Nil)
        }else{
            Nil
        }
    }
    case Cons(k,lst) => {
        if (f(k) == true){
            Cons(k,filterNumList(lst,f))
        }
        else{
            filterNumList(lst,f)
        }
    }
}

defined trait NumList
defined object Nil
defined class Cons
defined function filterNumList

In [15]:
val l1 = Cons(12, Cons(25, Cons(37, Nil)))
def f1(j: Int): Boolean =  j <= 25 && j >= 12
assert(filterNumList(l1, f1) == Cons(12, Cons(25, Nil)), "Test 1 failed.")


val l2 = Cons(22, Cons(135, Cons(137, l1)))
def f2(j: Int): Boolean =  j % 5 == 0
assert(filterNumList(l2, f2) == Cons(135, Cons(25, Nil)), "Test 2 failed.")

def f3(j: Int): Boolean =  j >= 210
assert(filterNumList(l2, f3) == Nil, "Test 3 failed.")
assert(filterNumList(Nil, f3) == Nil, "Test 4 failed.")

val l4 = Cons(0, Cons(0, Cons(0, Cons(0, Cons(0, Cons(0, Nil))))))

def f4(j: Int): Boolean =  j <= 0
assert(filterNumList(l4, f4) == l4, "Test 5 failed")

passed(8)


*** Tests Passed (8 points) ***


l1: Cons = Cons(12, Cons(25, Cons(37, Nil)))
defined function f1
l2: Cons = Cons(22, Cons(135, Cons(137, Cons(12, Cons(25, Cons(37, Nil))))))
defined function f2
defined function f3
l4: Cons = Cons(0, Cons(0, Cons(0, Cons(0, Cons(0, Cons(0, Nil))))))
defined function f4